# 产品关联分析
## （1）读取四个数据表格

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

order_df = pd.read_csv('data/order.csv', encoding='gbk')
product_df = pd.read_csv('data/product.csv', encoding='gbk')
customer_df = pd.read_csv('data/customer.csv', encoding='gbk')
date_df = pd.read_csv('data/date.csv', encoding='gbk')

In [3]:
order_df.head()

,订单日期,年份,订单数量,产品ID,客户ID,交易类型,销售区域ID,销售大区,国家,区域,产品类别,产品型号名称,产品名称,产品成本,利润,单价,销售金额
0,2016/1/1,2016,1,528,14432BA,1,4,西南区,中国,大中华区,配件,Rawlings Heart of THE Hide-11.5,棒球手套,500.0,1199.0,1699.0,1699.0
1,2016/1/2,2016,1,528,18741BA,1,4,西南区,中国,大中华区,配件,Rawlings Heart of THE Hide-11.5,棒球手套,500.0,1199.0,1699.0,1699.0
2,2016/1/2,2016,1,528,27988BA,1,4,西南区,中国,大中华区,配件,Rawlings Heart of THE Hide-11.5,棒球手套,500.0,1199.0,1699.0,1699.0
3,2016/1/5,2016,1,528,25710BA,1,4,西南区,中国,大中华区,配件,Rawlings Heart of THE Hide-11.5,棒球手套,500.0,1199.0,1699.0,1699.0
4,2016/1/6,2016,1,528,14999BA,1,4,西南区,中国,大中华区,配件,Rawlings Heart of THE Hide-11.5,棒球手套,500.0,1199.0,1699.0,1699.0


In [4]:
product_df.head()

,产品类别,产品ID,产品型号,产品名称
0,配件,528,Rawlings Heart of THE Hide-11.5,棒球手套
1,配件,480,Rawlings Gold Glove-11.5,棒球手套
2,配件,537,Mizuno MVP-12,棒球手套
3,配件,529,Wilson-A2000-12.5,棒球手套
4,配件,536,Rawlings Pro Preffered-13-FirstBaseMitt,棒球手套


In [5]:
customer_df.head()

,客户ID
0,30410BA
1,23789BA
2,27884BA
3,16522BA
4,13024BA


In [6]:
date_df.head()

,日期,年度,季度,月份,日,年度季度,年度月份,星期几
0,2013/7/1,2013,Q3,7,1,2013Q3,201307,1
1,2013/7/2,2013,Q3,7,2,2013Q3,201307,2
2,2013/7/3,2013,Q3,7,3,2013Q3,201307,3
3,2013/7/4,2013,Q3,7,4,2013Q3,201307,4
4,2013/7/5,2013,Q3,7,5,2013Q3,201307,5


## （2）定义事项
生成关联规则的目的是为了构建推荐系统，即向购买了某一组商品A的用户推荐另一组商品B。推荐的根据就是从历史数据来看，购买了A组商品的用户也会购买B组商品。

在关联分析中，第一件要完成的事情就是如何定义事项，即何种条件下算是一组商品被同时购买。一般的情况是归结在同一个订单号下的一组商品算是一个事项（item），在该问题中，数据集中没有订单编号，那么可以认为被同一个用户购买的一组商品算是一个事项。所以我们接下来从订单数据集中抽取出全部的事项，构成事项集。

In [10]:
item_set = [list(value) for key, value in order_df.groupby('客户ID')['产品ID']]
item_set[:5]

[[573, 485, 488, 214, 353, 541, 530, 344],
 [350, 604, 478, 479, 477, 477, 353, 485, 225, 491, 217],
 [346, 561, 222, 359],
 [564, 225, 361, 478, 477, 541, 530, 480, 346],
 [345, 355, 562, 485, 217, 214]]

In [11]:
def create_C1(data_set):
    """ 
    通过扫描整个事项集来创建一阶频繁项候选集合C1
    Args:
        data_set: List[Set]
                由事项构成的一个列表，每一个事项就是一组项的集合，可以
                将它想象为本次消费的一组商品
    Returns:
        C1: Set[frozonset]
            一阶频繁项候选集合
    """
    # 初始化一阶候选集
    C1 = set()
    # 将事项集中的每一个事项中项都取出来，放入到一阶候选集中
    # 一级循环：将事项集中的每一个事项都取出来
    for t in data_set:
        # 二级循环：将事项中的每一个项都取出来
        for item in t:
            # 单一的项先转换为锁定集合
            # 所谓的锁定集合frozenset就是生成后就不允许再改动其中元素的集合
            item_set = frozenset([item])
            # 将这个锁定项集添加到一阶候选集中
            # 由于C1是一个集合，所以重复的元素会自动进行移除
            C1.add(item_set)
    return C1


def is_apriori(Ck_item, Lksub1):
    """ 
    判断一个K阶项集的全部K-1阶子项集是否为K-1阶的频繁项集
    
    Args:
        Ck_item: frozenset
            一个K阶项集
        Lksub1: Set[frozenset]
            K-1阶频繁项集
    Return: bool
        如果结果成立的话，就返回True
        否则返回False
    """
    # 这里面的基本原理就是从K阶项集中移除某一个元素之后，得到了
    # 一个K-1阶项集，在判断这个K-1阶项集是否在K-1阶频繁项集中。
    # 这样就可以遍历全部的K-1阶项集。
    for item in Ck_item:
        sub_Ck = Ck_item - frozenset([item])
        if sub_Ck not in Lksub1:
            return False
    return True


def create_Ck(Lksub1, k):
    """
    基于K-1阶频繁项集去构建K阶频繁项集
    Args:
        Lksub1: Set[frozonset]
            K-1阶频繁项集
        k: int
            目标频繁项阶数
    Return: Set[frozonset]
        Ck: K阶频繁项候选集
    """
    # 初始化空的k阶频繁项候选集
    Ck = set()
    # 目前的k-1阶频繁项集的数量
    len_Lksub1 = len(Lksub1)
    # 将k-1阶频繁项集从集合set转换为列表list
    list_Lksub1 = list(Lksub1)
    # 一级循环：将K-1阶频繁项集的索引从0开始迭代处理，到最后一个为止，不包括最后一个
    for i in range(len_Lksub1-1):
        # 二级循环：从i的下一个索引开始处理
        for j in range(i+1, len_Lksub1):
            # 将frozonset转换为list
            l1 = list(list_Lksub1[i])
            l2 = list(list_Lksub1[j])
            # 将内部元素进行排序
            l1.sort()
            l2.sort()
            # 这个两个事项的前k-2个完全相同的话，就可以组成潜在的K阶频繁项候选了
            # 比如l1是{1, 2}， l2是{1, 3}，那么他们两个的第0号元素1相同，就有可能
            # 构成3阶频繁项集的候选{1, 2, 3}
            if l1[0:k-2] == l2[0:k-2]:
                Ck_item = list_Lksub1[i] | list_Lksub1[j]
                # 检验Ck_item中的每一个k-1子集是不是K-1频繁项集中的元素
                # 如果是的话，就加入到K阶频繁项候选集Ck中
                if is_apriori(Ck_item, Lksub1):
                    Ck.add(Ck_item)
    return Ck


def generate_Lk_by_Ck(data_set, Ck, min_support, support_data):
    """
    基于K阶频繁项候选集Ck生成K阶频繁项集Lk
    Args:
        data_set: List[Set]
            事项集
        Ck: Set[frozonset]
            K阶频繁项候选集
        min_support: float
            可以通过筛选的最小支持度（实际上就是出现的频率）
        support_data: dict
            记录了每一个频繁项的出现概率
    Returns:
        Lk: Set[frozonset]
            K阶频繁项集
    """
    # 初始化一个空的K阶频繁项集
    Lk = set()
    # 初始化一个用于记录每一个候选项出现次数的字典
    item_count = {}
    # 一级循环：将事项集中的每一个事项取出来进行处理
    for t in data_set:
        # 二级循环：将K阶频繁项候选集Ck中的每一个项取出来处理
        for item in Ck:
            # 如果候选项item是事务项t的子集，也就是事务项中有这一个候选项
            if item.issubset(t):
                # 如果这个候选项还没有出现在item_count中，就在计数字典中将该
                # 候选项的计数初始化为1
                if item not in item_count:
                    item_count[item] = 1
                # 如果已经出现在item_count中，就在计数字典中该候选项的计数+1
                else:
                    item_count[item] += 1
    # 计算出事项集中事项的数量
    t_num = float(len(data_set))
    # 将计数字典中的每一个候选项取出来进行迭代处理
    for item in item_count:
        # 如果该候选项item出现的频率（频数/总数）大于最小支持度min_support
        # 那么就把这个候选项添加到K阶频繁项集中，并记录这一个item的频率
        if (item_count[item] / t_num) >= min_support:
            Lk.add(item)
            support_data[item] = item_count[item] / t_num
    return Lk


def generate_L(data_set, k, min_support):
    """        
    生成全部的频繁项集
    Args:
        data_set: List[Set]
            事项集
        k: int
            频繁项集的最大阶数
        min_support: float
            频繁项集的最小支持度（也就是最小出现频率）
    Returns:
        L: List[Set[frozenset]]
            总体频繁项集
        support_data: Dict[frozenset, float]
            保存着每一个频繁项和其支持度的字典
    """
    support_data = {}
    # 产生一阶频繁项候选集合
    C1 = create_C1(data_set)
    # 从一阶频繁项候选集合中筛选出一阶频繁项集合
    L1 = generate_Lk_by_Ck(data_set, C1, min_support, support_data)
    # 从一阶频繁项集合复制一份出来
    Lksub1 = L1.copy()
    # 初始化一个总体频繁项集
    L = []
    # 将一阶频繁项集追加到总体频繁项集中
    L.append(Lksub1)
    # 从2到k+1(不包括k+1)，依次处理
    for i in range(2, k+1):
        # 基于i-1阶频繁项集产生i阶频繁项候选集
        Ci = create_Ck(Lksub1, i)
        # 从i阶频繁项候选集中筛选出i阶频繁项集
        Li = generate_Lk_by_Ck(data_set, Ci, min_support, support_data)
        # 复制一份
        Lksub1 = Li.copy()
        # 添加到总体频繁项集中
        L.append(Lksub1)
    return L, support_data


## （3）检索出全部的频繁项集和相应的支持度

In [22]:
L, support_data = generate_L(item_set, k=6, min_support=0.01)

这里我们要求频繁项集的最高阶数为6，最小的支持度为0.01，也就是说每一个频繁项出现的频率不低于1%。结果如下：

In [24]:
L

[{frozenset({606}),
  frozenset({359}),
  frozenset({485}),
  frozenset({362}),
  frozenset({483}),
  frozenset({463}),
  frozenset({580}),
  frozenset({312}),
  frozenset({384}),
  frozenset({582}),
  frozenset({540}),
  frozenset({237}),
  frozenset({491}),
  frozenset({360}),
  frozenset({313}),
  frozenset({228}),
  frozenset({484}),
  frozenset({314}),
  frozenset({541}),
  frozenset({386}),
  frozenset({475}),
  frozenset({486}),
  frozenset({535}),
  frozenset({467}),
  frozenset({225}),
  frozenset({214}),
  frozenset({481}),
  frozenset({583}),
  frozenset({529}),
  frozenset({476}),
  frozenset({530}),
  frozenset({477}),
  frozenset({536}),
  frozenset({489}),
  frozenset({478}),
  frozenset({222}),
  frozenset({487}),
  frozenset({357}),
  frozenset({490}),
  frozenset({355}),
  frozenset({480}),
  frozenset({234}),
  frozenset({231}),
  frozenset({584}),
  frozenset({537}),
  frozenset({538}),
  frozenset({528}),
  frozenset({482}),
  frozenset({382}),
  frozenset({465}),


这么来看，只找到了1,2,3阶的频繁项，更高阶的项出现的频率达不到1%的支持度。
## 基于频繁项确定关联规则


In [35]:
def generate_big_rules(L, support_data, min_conf):
    """                
    基于总体频繁项集，生成关联规则集
    
    Args:
        L: List[Set[frozenset]]
            总体频繁项集
        support_data: Dict[frozenset, float]
            频繁项-->支持度字典
        min_conf: float
            最低置信度
    Returns:
        big_rule_list: List[Tuple[frozenset, frozenset, float]]
            关联规则集合
    """
    big_rule_list = []
    sub_set_list = []
    # 一级循环：依次处理第1阶频繁项集和第K阶频繁项集
    for i in range(0, len(L)):
        # 二级循环：依次处理第i阶频繁项集中的每一个频繁项
        for freq_set in L[i]:
            # 三级循环：依次将低阶频繁项与该频繁项进行对比，计算出该频繁项对
            # 每一个低阶频繁项的置信度
            for sub_set in sub_set_list:
                # 如果低阶频繁项sub_set是当前频繁项freq_set的子集
                # 记低阶频繁项为B，低阶频繁项的补集为A，那么当前的频繁项就是A+B
                if sub_set.issubset(freq_set):
                    # 计算出Support(A+B) / Support(A)的结果
                    # 因为A是包含项更少的子集，所以更容易出现，支持度更高
                    # 而A+B的支持度就会低很多
                    # 这个计算结果就是在观测到A的条件下，进一步观察到A和B同时出现的概率
                    # 如果这个概率很大，就说明了只要观测到A，就可以断定B也会出现
                    # 即A--->B就是一条关联规则。
                    conf = support_data[freq_set] / support_data[freq_set - sub_set]
                    # 将关联规则记录为(A, B, 关联得分)
                    big_rule = (freq_set - sub_set, sub_set, conf)
                    # 如果关联得分超过一定的阈值且之前没有记录过，就把这个规则记录下来
                    if conf >= min_conf and big_rule not in big_rule_list:
                        big_rule_list.append(big_rule)
            # 处理完该频繁项之后，就把它们添加到低阶频繁项中，因为在下一轮循环中
            # 它们就自然变成了低阶频繁项
            sub_set_list.append(freq_set)
    return big_rule_list

big_rule_list = generate_big_rules(L, support_data, min_conf=0.5)
for i in big_rule_list:
    print('%s -- > %s: %.2f %%' %(list(i[0]), list(i[1]), i[2]*100))

[535] -- > [528]: 60.60 %
[478] -- > [477]: 85.03 %
[479] -- > [477]: 89.53 %
[536] -- > [528]: 70.20 %
[530] -- > [541]: 56.70 %
[541] -- > [530]: 86.96 %
[538] -- > [529]: 54.85 %
[540] -- > [529]: 70.37 %
[537] -- > [528]: 71.70 %
[539] -- > [529]: 67.79 %
[217, 537] -- > [528]: 88.60 %
[222, 478] -- > [477]: 87.68 %
[477, 222] -- > [478]: 52.31 %
[217, 478] -- > [477]: 83.33 %
[217, 477] -- > [478]: 51.84 %
[537, 478] -- > [477]: 93.55 %
[537, 477] -- > [478]: 70.00 %
[480, 536] -- > [528]: 64.90 %
[225, 478] -- > [477]: 93.16 %
[537, 222] -- > [528]: 86.15 %
[528, 477] -- > [537]: 68.86 %
[537, 477] -- > [528]: 79.31 %
[537, 214] -- > [528]: 82.73 %
[214, 478] -- > [477]: 85.81 %
[477, 214] -- > [478]: 50.19 %
[480, 535] -- > [528]: 58.47 %
[480, 537] -- > [528]: 69.61 %
[485, 478] -- > [477]: 84.87 %
[477, 485] -- > [478]: 86.77 %
[214, 479] -- > [477]: 96.53 %
[480, 540] -- > [529]: 71.05 %
[480, 530] -- > [541]: 57.81 %
[480, 541] -- > [530]: 92.50 %
[480, 478] -- > [477]: 91.5

至此，我们就完成了对订单记录中的关联规则的发现。